# Single Table Modeling

**SDV** has special support for modeling single table datasets using a variety of models.

Currently, SDV implements:

* GaussianCopula: A tool to model multivariate distributions using [copula functions](https://en.wikipedia.org/wiki/Copula_%28probability_theory%29). Based on our [Copulas Library](https://github.com/sdv-dev/Copulas).
* CTGAN: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

## GaussianCopula

In this first part of the tutorial we will be using the GaussianCopula class to model the `users` table
from the toy dataset included in the **SDV** library.

### 1. Load the Data

In [1]:
from sdv import load_demo

users = load_demo()['users']

This will return a table with 4 fields:

* `user_id`: A unique identifier of the user.
* `country`: A 2 letter code of the country of residence of the user.
* `gender`: A single letter code, `M` or `F`, indicating the user gender. Note that this demo simulates the case where some users did not indicate the gender, which resulted in empty data values in some rows.
* `age`: The age of the user, in years.

In [2]:
users

,user_id,country,gender,age
0,0,USA,M,34
1,1,UK,F,23
2,2,ES,None,44
3,3,UK,M,22
4,4,USA,F,54
5,5,DE,M,57
6,6,BG,F,45
7,7,ES,None,41
8,8,FR,F,23
9,9,UK,None,30


### 2. Prepare the model

In order to properly model our data we will need to provide some additional information to our model,
so let's prepare this information in some variables.

First, let's indicate that the `user_id` field in our table is the primary key, so we do not want our
model to attempt to learn it.

In [3]:
primary_key = 'user_id'

We will also want to anonymize the countries of residence of our users, to avoid disclosing such information.
Let's make a variable indicating that the `country` field needs to be anonymized using fake `country_codes`.

In [4]:
anonymize_fileds = {
    'country': 'contry_code'
}

The full list of categories supported corresponds to the `Faker` library
[provider names](https://faker.readthedocs.io/en/master/providers.html)

Once we have prepared the arguments for our model we are ready to import it, create an instance
and fit it to our data.

In [8]:
from sdv.tabular import GaussianCopula

model = GaussianCopula(
    primary_key=primary_key,
    anonymize_fileds=anonymize_fileds
)
model.fit(users)

2020-07-09 20:41:29,959 - INFO - table - Loading transformer CategoricalTransformer for field country
2020-07-09 20:41:29,961 - INFO - table - Loading transformer CategoricalTransformer for field gender
2020-07-09 20:41:29,962 - INFO - table - Loading transformer NumericalTransformer for field age
2020-07-09 20:41:29,981 - INFO - gaussian - Fitting GaussianMultivariate(distribution="GaussianUnivariate")


**Notice** how the model took care of transforming the different fields using the appropriate
Reversible Data Transforms to ensure that the data has a format that the GaussianMultivariate model
from the [copulas](https://github.com/sdv-dev/Copulas) library can handle.

### 3. Sample data from the fitted model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from our model.

In [10]:
sampled = model.sample()

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [11]:
sampled

,user_id,country,gender,age
0,0,USA,F,29
1,1,FR,F,44
2,2,UK,M,38
3,3,ES,M,19
4,4,USA,F,55
5,5,USA,NaN,27
6,6,USA,F,27
7,7,UK,NaN,7
8,8,FR,F,55
9,9,UK,NaN,43


Notice, as well that the number of rows generated by default corresponds to the number of rows that
the original table had, but that this number can be changed by simply passing it:

In [12]:
model.sample(5)

,user_id,country,gender,age
0,0,BG,M,30
1,1,USA,NaN,61
2,2,USA,M,35
3,3,ES,NaN,34
4,4,USA,F,41


## CTGAN

In this second part of the tutorial we will be using the CTGAN model to learn the data from the
demo dataset called `census`, which is based on the [UCI Adult Census Dataset]('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data').

### 1. Load the Data

In [13]:
from sdv import load_demo

census = load_demo('census')['census']

2020-07-09 20:48:49,870 - INFO - __init__ - Loading table census


This will return a table with several rows of multiple data types:

In [15]:
census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 2. Prepare the model

In this case there is no primary key to setup and we will not be anonymizing anything, so the only
thing that we will pass to the CTGAN model is the number of epochs that we want it to perform when
it leanrs the data, which we will keep low to make this execution quick.

In [16]:
from sdv.tabular import CTGAN

model = CTGAN(epochs=10)

/home/xals/.virtualenvs/SDV/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


Once the instance is created, we can fit it to our data. Bear in mind that this process might take some
time to finish, especially on non-GPU enabled systems.

In [18]:
model.fit(census)

2020-07-09 20:51:21,668 - INFO - table - Loading transformer NumericalTransformer for field age
2020-07-09 20:51:21,669 - INFO - table - Loading transformer LabelEncodingTransformer for field workclass
2020-07-09 20:51:21,670 - INFO - table - Loading transformer NumericalTransformer for field fnlwgt
2020-07-09 20:51:21,670 - INFO - table - Loading transformer LabelEncodingTransformer for field education
2020-07-09 20:51:21,671 - INFO - table - Loading transformer NumericalTransformer for field education-num
2020-07-09 20:51:21,672 - INFO - table - Loading transformer LabelEncodingTransformer for field marital-status
2020-07-09 20:51:21,672 - INFO - table - Loading transformer LabelEncodingTransformer for field occupation
2020-07-09 20:51:21,673 - INFO - table - Loading transformer LabelEncodingTransformer for field relationship
2020-07-09 20:51:21,673 - INFO - table - Loading transformer LabelEncodingTransformer for field race
2020-07-09 20:51:21,674 - INFO - table - Loading transforme

Epoch 1, Loss G: 1.9379, Loss D: -0.1297
Epoch 2, Loss G: 1.6529, Loss D: 0.1716
Epoch 3, Loss G: 1.0939, Loss D: 0.0208
Epoch 4, Loss G: 0.9390, Loss D: -0.0918
Epoch 5, Loss G: -0.0550, Loss D: 0.0415
Epoch 6, Loss G: -0.0864, Loss D: -0.0104
Epoch 7, Loss G: -0.3865, Loss D: 0.0074
Epoch 8, Loss G: -1.0150, Loss D: -0.0708
Epoch 9, Loss G: -0.8685, Loss D: -0.1289
Epoch 10, Loss G: -1.1524, Loss D: -0.0487


### 3. Sample data from the fitted model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from our model just like we did with the GaussianCopula model.

In [23]:
sampled = model.sample()

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [24]:
sampled.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,38,Private,212843,Prof-school,10,Never-married,Tech-support,Unmarried,White,Male,49,-1,40,United-States,<=50K
1,30,Federal-gov,321819,HS-grad,14,Married-civ-spouse,Craft-repair,Unmarried,Black,Female,51,-3,39,United-States,<=50K
2,25,Private,169771,HS-grad,13,Never-married,Other-service,Not-in-family,White,Female,-37,-2,39,United-States,<=50K
3,47,Private,116751,Some-college,6,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,2057,41,United-States,<=50K
4,38,Private,315119,Assoc-acdm,13,Never-married,Craft-repair,Husband,Black,Male,-11,-1,49,United-States,<=50K
5,36,State-gov,172646,Masters,6,Married-civ-spouse,Machine-op-inspct,Wife,White,Male,-50,-2,40,Japan,<=50K
6,40,Private,163368,Some-college,4,Married-civ-spouse,Transport-moving,Own-child,White,Female,-13,5,39,United-States,<=50K
7,23,Private,369324,Some-college,11,Married-civ-spouse,Tech-support,Husband,White,Female,-60,5,40,United-States,<=50K
8,32,Private,192521,Bachelors,10,Married-civ-spouse,Exec-managerial,Not-in-family,White,Female,-15,2,43,United-States,<=50K
9,28,Private,244118,HS-grad,9,Separated,Prof-specialty,Not-in-family,White,Male,-21,0,39,United-States,>50K


### 4. Evaluate how good the data is

Finally, we will use the evaluation framework included in SDV to obtain a metric of how
similar the sampled data is to the original one.

For this, we will simply import the `sdv.evaluation.evaluate` function and pass both
the synthetic and the real data to it.

In [25]:
import warnings
warnings.filterwarnings('ignore')

from sdv.evaluation import evaluate

evaluate(sampled, census)

-28.22245129315498